In [51]:
from dotenv import load_dotenv
from docx_parser import getDoc
from docx_parser import getCsv
from langchain import FewShotPromptTemplate, PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain
import os

In [52]:
# Cargo variables de configuración
load_dotenv()

True

In [53]:
# Carga documento input y tabla objetivo.
#context_file1 = 'documentos/migraciones/cp_migraciones.docx'
#context1 = getDoc(context_file1)

#targte_file1 = 'documentos/migraciones/objetivos_migraciones.csv'
#target1 = getCsv(targte_file1)

# Carga documento input y tabla objetivo.
context_file2 = 'documentos/redenciones/cp_redenciones.docx'
context2 = getDoc(context_file2)

targte_file2 = 'documentos/redenciones/objetivos_redenciones.csv'
target2 = getCsv(targte_file2)

# Carga documento input y tabla objetivo.
#context_file3 = 'documentos/msi/cp_msi.docx'
#context3 = getDoc(context_file3)

#targte_file3 = 'documentos/msi/objetivos_msi.csv'
#target3 = getCsv(targte_file3)

In [54]:
prompt = """You will be provided with a context from which you should extract multiple test cases by identifying the following aspects:
 'Caso de prueba', 'Modulo', 'Funcionalidad', 'Proceso' and 'Segmento'.
   As the text contains multiple test cases, the output should be presented in a csv format table.
   Here is an example:"""

In [55]:
examples = [
    {
        "context": context2,
        "result": target2
    } 
]

In [36]:
"""examples = [
    {
        "context": context1,
        "result": target1
    },
    {
        "context": context2,
        "result": target2
    },
    {
        "context": context3,
        "result": target3
    }
]"""

'examples = [\n    {\n        "context": context1,\n        "result": target1\n    },\n    {\n        "context": context2,\n        "result": target2\n    },\n    {\n        "context": context3,\n        "result": target3\n    }\n]'

In [56]:
example_template = """
Context: {context}
Result: {result}
"""


example_prompt = PromptTemplate(
    input_variables=["context", "result"],
    template=example_template
)


prefix = """Sos parte del equipo de testing re una compania de telecomunicaciones.
Te voy a dar un documento con requerimientos de testeo para uno de los modulos de la aplicacion.
Necesito que hagas una lista con todos los casos de prueba que puedas identificar.
 Deberás incluir la condicion esperada para cada uno de los casos.
    
   Aqui hay ejemplos:
   """

suffix = """
Context: {new_context}
Result: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["new_context"],
    example_separator="\n\n"
)


In [57]:
#contexto_path = 'documentos/migraciones/hu_migraciones.docx'
contexto_path = 'documentos/redenciones/hu_redenciones.docx'
#contexto_path = 'documentos/msi/hu_msi.docx'
nuevo_contexto = getDoc(contexto_path)
#print(nuevo_contexto)

In [58]:
llm = AzureChatOpenAI (
        azure_endpoint=os.getenv('API_BASE'),
        openai_api_version=os.getenv('API_VERSION'),
        deployment_name=os.getenv('CHAT_ENGINE_16K'),
        openai_api_key=os.getenv('API_KEY'),
        openai_api_type=os.getenv('API_TYPE'),
        temperature=0,
        model_version="0613" # es importante aclararlo para el correcto calculo de los costos
        )

llm_chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

result = llm_chain(nuevo_contexto)

In [59]:
result['text']

'MÓDULO ;FUNCIONALIDAD;PROCESO;SEGMENTO;CASO DE PRUEBA\nRedenciónes;Mantenedor - Bonos, Descuentos y Regalos;Ingresar bono, descuento o regalo;Control y pospago;\nRedenciónes;Mantenedor - Bonos, Descuentos y Regalos;Seleccionar bono, descuento o regalo;Control y pospago;\nRedenciónes;Mantenedor - Bonos, Descuentos y Regalos;Visualizar bono, descuento o regalo seleccionado;Control y pospago;\nRedenciónes;Mantenedor - Bonos, Descuentos y Regalos;Eliminar bono, descuento o regalo seleccionado;Control y pospago;\nRedenciónes;Mantenedor - Bonos, Descuentos y Regalos;Ingresar imagen para bono, descuento o regalo;Control y pospago;\nRedenciónes;Mantenedor - Campaña Fidelización;Configurar regalo por antigüedad del cliente;Control y pospago;\nRedenciónes;Mantenedor - Campaña Fidelización;Configurar vigencia de visualización del regalo;Control y pospago;\nRedenciónes;Mantenedor - Campaña Fidelización;Ingresar bono, descuento o regalo para la campaña;Control y pospago;\nRedenciónes;Mantenedor - 